In [8]:
import shapely
from utils.vector2shape import reverse_vector_polygon
from main_finetune import get_finetune_dataset_mnist
import torch

In [4]:
train_data_path = "dataset/mnist_polygon_train_10k.csv"
train_tokens, train_labels, val_tokens, val_labels  = get_finetune_dataset_mnist(train_data_path, dataset_size=None, train=True)

In [5]:
test_dataset = "dataset/mnist_polygon_test_2k.npz"
test_tokens, test_labels = get_finetune_dataset_mnist(file=test_dataset, train=False)

In [6]:
train_flag = [1] * train_tokens.shape[0]
test_flag = [0] * test_tokens.shape[0]
valid_flag = [-1] * val_tokens.shape[0]

In [14]:
tokens = torch.cat([train_tokens, test_tokens, val_tokens], dim=0)
labels = torch.cat([train_labels, test_labels, val_labels], dim=0)
flags = train_flag + test_flag + valid_flag

In [15]:
polys = [reverse_vector_polygon(one) for one in tokens]

In [24]:
from utils.vectorizer import num_points_from_wkt
for one in polys:
    if num_points_from_wkt(one.wkt) != 64:
        print("A")

In [25]:
ids = list(range(1, len(polys)+1))
coors = tokens[:, :, :2].contiguous()

In [40]:
import numpy as np

a = polys[0]
xy = a.interiors[0].coords.xy
np.array(xy).T.shape

(19, 2)

In [50]:
pgon_list = []
for pgon in polys:
    ex_coors = np.expand_dims(np.array(pgon.exterior.coords), axis = 0)
    if len(pgon.interiors):
        in_coors = np.concatenate([np.array(interior.coords.xy).T for interior in pgon.interiors], axis = 0)
        in_coors = np.expand_dims(in_coors, axis = 0)
        coors = np.concatenate([in_coors, ex_coors], axis = 1)
    else:
        coors = ex_coors
    pgon_list.append(coors)
pgon_list = np.concatenate(pgon_list, axis = 0)

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame({"ID": ids, "TYPEID": labels, "SPLIT_0": flags, "geometry_norm": polys, "geom_coors": coors})

In [21]:
gdf.to_pickle("dataset/mnist_resnet1d_12k.pkl")

In [ ]:
# Simulate launching multiple different jobs that log to the same experiment

import wandb
import math
import random

for i in range(5):
  job_type = "rollout"
  if i == 2:
    job_type = "eval"
  if i == 3:
    job_type = "eval2"
  if i == 4:
    job_type = "optimizer"

  # Set group and job_type to see auto-grouping in the UI
  wandb.init(project="group-demo", 
             group="exp_" + str(1),
             entity="kangisve",
             job_type=job_type)

  for j in range(100):
    acc = 0.1 * (math.log(1 + j + .1) + random.random())
    val_acc = 0.1 * (math.log(1+ j + 2) + random.random() + random.random())
    if (j % 10 == 0):
      wandb.log({"acc":acc, "val_acc":val_acc})
  
  # Using this to mark a run complete in a notebook context
  wandb.finish()

In [2]:
from potae import PoTAE

pot = PoTAE(num_layers=3)
total_params_pot_model = sum(p.numel() for p in pot.parameters())
print(f"Total number of parameters in the Transformer model: {total_params_pot_model}")

Total number of parameters in the Transformer model: 218807
